# Day 5 - Lab 2: Plan-and-Execute & Multi-Agent Systems (Solution)

**Objective:** Explore advanced agent architectures, including the plan-and-execute model (SMOL-style) and conversational multi-agent systems using Microsoft's AutoGen.

**Introduction:**
This solution notebook provides the complete code and explanations for Lab 5.2. It covers the SMOL-style plan-and-execute pattern and demonstrates how to build and manage a team of collaborative AI agents with AutoGen.

## Step 1: Setup

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    # This works when running as a script
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # This works when running in an interactive environment (like a notebook)
    # We go up two levels from the notebook's directory to the project root.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
# This helper will install packages if they are not found
import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        %pip install -q {package}

install_if_missing('pyautogen')

from utils import setup_llm_client, get_completion, save_artifact
import autogen
import os

# We will use the OpenAI provider for this lab as AutoGen is optimized for it.
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): A SMOL-style Plan-and-Execute Agent

**Explanation:**
This pattern separates the creative, high-level thinking from the detailed, low-level implementation. 
1.  **Planner Agent:** The first prompt asks the LLM to act as an architect. Its only job is to think and create a detailed plan or specification. This focuses the LLM's attention on getting the logic and structure right without worrying about code syntax.
2.  **Coder Agent:** The second prompt is highly constrained. It receives the detailed spec from the planner and is instructed to *only* write the code that implements that spec. By giving it a clear plan to follow, we reduce the chances of the LLM misunderstanding the requirements or generating incorrect code.

In [ ]:
high_level_goal = "Create a Python function that takes a list of strings and returns a new list containing only the strings that are palindromes."

# 1. Write the prompt for the Planner Agent.
planner_prompt = f"""
You are a senior software architect. Your task is to create a detailed specification for a Python function based on the following goal.

**Goal:** {high_level_goal}

Provide a specification that includes:
- The function signature with type hints.
- A clear description of what the function does.
- A step-by-step description of the implementation logic.
"""

print("--- Planner Agent Generating Spec ---")
function_spec = get_completion(planner_prompt, client, model_name, api_provider)
print(function_spec)

# 2. Write the prompt for the Coder Agent.
coder_prompt = f"""
You are a Python developer. Your task is to write the code for a function based ONLY on the following specification.

**Specification:**
<spec>
{function_spec}
</spec>

Only output the raw Python code for the function. Do not include any explanation or example usage.
"""

print("\n--- Coder Agent Generating Code ---")
generated_function = get_completion(coder_prompt, client, model_name, api_provider)
print(generated_function)

### Challenge 2 (Intermediate): A Three-Agent AutoGen Team

**Explanation:**
AutoGen allows us to create multiple, specialized agents that collaborate through conversation.
1.  **`config_list`**: This tells AutoGen how to configure the LLM client. It's how we provide the model name and API key.
2.  **`UserProxyAgent`**: This special agent acts as the proxy for the human user. We set `human_input_mode` to `TERMINATE` to tell it that once a solution is found (i.e., the code is written), the conversation should end automatically without asking for human feedback.
3.  **`AssistantAgent`**: These are the general-purpose AI agents. We create two of them, giving each a unique `name` and a `system_message` that defines its role and personality.
4.  **`GroupChat`**: This object manages the conversation between the list of agents.
5.  **`GroupChatManager`**: This is an agent that orchestrates the group chat, deciding which agent should speak next.
6.  **`user_proxy.initiate_chat`**: This kicks off the conversation with an initial message.

In [ ]:
# 1. Define the config_list for the LLM.
config_list = [
    {
        'model': model_name,
        'api_key': os.getenv("OPENAI_API_KEY"),
    }
]

# 2. Create the UserProxyAgent.
user_proxy = autogen.UserProxyAgent(
    name="UserProxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=5,
    code_execution_config={"work_dir": "coding"} # Specify a directory for generated code
)

# 3. Create the ProductManager agent.
product_manager = autogen.AssistantAgent(
    name="ProductManager",
    system_message="You are a Product Manager. Your job is to clarify requirements and create a clear, actionable plan for the developer.",
    llm_config={"config_list": config_list}
)

# 4. Create the Developer agent.
developer = autogen.AssistantAgent(
    name="Developer",
    system_message="You are a senior Python developer. You write clean, efficient Python code based on the Product Manager's plan. You must include the code in a ```python block.",
    llm_config={"config_list": config_list}
)

# 5. Create the GroupChat and GroupChatManager.
groupchat = autogen.GroupChat(agents=[user_proxy, product_manager, developer], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list})

# 6. Initiate the chat with a feature request.
feature_request = "Add a feature to our API to calculate the complexity of a password based on length, and the presence of uppercase, lowercase, numbers, and symbols."
user_proxy.initiate_chat(manager, message=feature_request)

### Challenge 3 (Advanced): Multi-Agent System with a Code Reviewer

**Explanation:**
This challenge makes our agent team more realistic by adding a QA step. 
1.  **`CodeReviewer` Agent:** We create a fourth agent with a specific persona: to review code for quality and correctness.
2.  **`is_termination_msg`**: This is the key to creating the review loop. We customize the `UserProxyAgent` by providing it with a lambda function. This function checks every incoming message. If the message content contains the string "APPROVED", the function returns `True`, which tells the `UserProxyAgent` that the task is complete and the conversation can end. If it doesn't find "APPROVED", it returns `False`, and the conversation continues, allowing the developer to address the reviewer's feedback.

In [ ]:
# 1. Create the CodeReviewer agent.
code_reviewer = autogen.AssistantAgent(
    name="CodeReviewer",
    system_message="You are a Code Reviewer. Your job is to inspect Python code for quality, correctness, and best practices. You must point out any issues. If the code is perfect, you must respond with only the word 'APPROVED'.",
    llm_config={"config_list": config_list}
)

# 2. Create a new UserProxyAgent with a custom termination message check.
user_proxy_with_review = autogen.UserProxyAgent(
    name="UserProxyWithReview",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    # This lambda function checks if the last message contains 'APPROVED'
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("APPROVED")
)

# 3. Create the new 4-agent GroupChat and Manager.
four_agent_groupchat = autogen.GroupChat(
    agents=[user_proxy_with_review, product_manager, developer, code_reviewer], 
    messages=[], 
    max_round=15
)
four_agent_manager = autogen.GroupChatManager(groupchat=four_agent_groupchat, llm_config={"config_list": config_list})

# 4. Initiate the chat.
user_proxy_with_review.initiate_chat(four_agent_manager, message=feature_request)

## Lab Conclusion

Excellent work! You have now explored two powerful advanced agentic architectures. You learned how the plan-and-execute model can lead to more structured and reliable code generation, and you used AutoGen to simulate a collaborative team of AI agents that can plan, code, and review work. These foundational patterns are the building blocks for creating highly sophisticated and autonomous AI systems.